In [1]:
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json

#df_annotations = pd.read_csv("../../data/ctgov/annotations/all_annotations.csv")


# Get data

In [2]:
# with open("../../data/ctgov/protocols.pkl.gz", "rb") as f:
#     file_to_pages = pkl.load(f)

In [3]:
# import spacy
# nlp = spacy.blank("en")

In [4]:
# texts = []

# for i in range(len(df_annotations)):
#     file_name = df_annotations.file.iloc[i]
    
#     pages= file_to_pages[file_name]
    
#     text = ""
#     for page in pages:
#         doc = nlp(page)
        
#         is_include = [False] * len(doc)
#         for tok in doc:
#             if tok.text.lower() in {"arm", "arms", "armed", "cohort", "cohorts", "group", "groups"}:
#                 for token_index in range(tok.i - 20, tok.i + 20):
#                     if token_index >= 0 and token_index < len(doc):
#                         is_include[token_index] = True
        
#         for token_index in range(len(doc)):
#             if is_include[token_index]:
#                 text += doc[token_index].text + doc[token_index].whitespace_
                
#     if text == "":
#         print ("nothing found", file_name, df_annotations.num_arms.iloc[i])
#     else:
#         print ("found", file_name, df_annotations.num_arms.iloc[i])

#     texts.append(text)
# df_annotations["text"] = texts

In [5]:
# df_annotations.to_pickle("filtered_for_arm.pkl")

In [6]:
df_annotations = pd.read_csv("filtered_for_arm.pkl.csv.bz2")

In [7]:
# del file_to_pages

In [8]:
df_annotations_num_arms_known = df_annotations[~df_annotations.num_arms.isna() & ~df_annotations.text.isna()]

In [9]:
#df_annotations_num_arms_known = df_annotations_num_arms_known.sample(1000)

In [10]:
def get_num_arms_clean(num):
    if num >= 5:
        num = 5
    return num
df_annotations_num_arms_known["num_arms_clean"] = df_annotations_num_arms_known["num_arms"].apply(get_num_arms_clean)

/tmp/ipykernel_17469/3303076186.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_annotations_num_arms_known["num_arms_clean"] = df_annotations_num_arms_known["num_arms"].apply(get_num_arms_clean)


In [11]:
df_train = df_annotations_num_arms_known[df_annotations_num_arms_known.train_val == "train"]

In [12]:
# TRAINING_DATA = [
#     ["My little kitty is so special", {"KAT0": True}],
#     ["Dude, Totally, Yeah, Video Games", {"KAT1": True}],
#     ["Should I pay $1,000 for the iPhone X?", {"KAT1": True}],
#     ["The iPhone 8 reviews are here", {"KAT1": True}],
#     ["Noa is a great cat name.", {"KAT0": True}],
#     ["We got a new kitten!", {"KAT0": True}]
# ]

TRAINING_DATA = []
for idx in range(len(df_train)):
    cats = {}
    for a in range(1, 6):
        cats[str(a)] = 0
    cats[str(int(df_train.num_arms.iloc[idx]))] = 1
    text = df_train.text.iloc[idx]
    if len(text) > 1000000:
        text = text[:1000000]
    
    TRAINING_DATA.append([text , cats])

In [ ]:
import spacy
# Add imports for example, as well as textcat config...
from spacy.training import Example
from spacy.pipeline.textcat import single_label_bow_config, single_label_default_config
from thinc.api import Config
import random

# labels should be one-hot encoded



# bow
# config = Config().from_str(single_label_bow_config)

# textensemble with attention
config = Config().from_str(single_label_default_config)

nlp = spacy.blank("en")
# now uses `add_pipe` instead
category = nlp.add_pipe("textcat", last=True, config=config)
for a in range(1, 6):
    category.add_label(str(a))


# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(100):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=4):
        texts = [nlp.make_doc(text) for text, entities in batch]
        annotations = [{"cats": entities} for text, entities in batch]

        # uses an example object rather than text/annotation tuple
        examples = [Example.from_dict(doc, annotation) for doc, annotation in zip(
            texts, annotations
        )]
        nlp.update(examples, losses=losses)
    if itn % 20 == 0:
        print(losses)


2022-11-13 12:37:56.488795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 12:37:56.646019: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-13 12:37:56.646038: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-13 12:37:56.674430: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-13 12:37:58.990246: W tensorflow/stream_executor/platform/de

In [ ]:
df_annotations_num_arms_known = df_annotations[~df_annotations.num_arms.isna()]

In [ ]:
df_annotations_num_arms_known["num_arms_clean"] = df_annotations_num_arms_known["num_arms"].apply(get_num_arms_clean)

In [ ]:
df_val = df_annotations_num_arms_known[df_annotations_num_arms_known.train_val == "val"]

In [ ]:
predictions = []
for idx in range(len(df_val)):
    doc = nlp(df_val.text.apply(str).iloc[idx])
    predictions.append(doc.cats)

In [ ]:
df_val["y_pred"] = [int(max(p, key=p.get)) for p in predictions]

In [ ]:
num_arms_ground_truths = df_val.num_arms_clean
pred_num_arms = df_val["y_pred"]

In [ ]:
acc = accuracy_score(num_arms_ground_truths, pred_num_arms)
print (f"Num arms accuracy {acc}")

In [ ]:
ConfusionMatrixDisplay.from_predictions(num_arms_ground_truths, pred_num_arms)
plt.xticks(rotation=90)
;

In [ ]:
nlp.to_disk("arms_model_14_textcat_numarms.spacy")
